<a href="https://colab.research.google.com/github/SophieShin/MachineLearning/blob/main/Option_Pricing(%2Bprc_lmt).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### 반드시 사본 저장 후 사본파일에서 실행해주세요.(파일/Drive에 사본저장)
#### 아래 '재생'버튼을 누르면서 진행하면 됩니다


In [1]:
# 필요한 라이브러리 import
import pandas as pd
import numpy as np

#### [CR20]화면에서 검증하고자 하는 상품의 이론가격 파라미터를 저장합니다
#### 결제월 : 전체
#### 선물옵션구분: 콜+풋
#### 이론가용도: 기준가용
#### 화면에서 프린트 버튼을 누른 뒤 엑셀로 저장하면 됩니다.
#### 엑셀로 저장할 때  'CSV' 형식으로 저장합니다.

In [135]:
# 파일 업로드
from google.colab import files

uploaded = files.upload()

Saving 상하한가.csv to 상하한가 (3).csv


#### 아래 '이론가파람_SK하이닉스' 부분을 자신이 저장한 파일 이름으로 변경합니다

In [192]:
import io
df = pd.read_csv(io.BytesIO(uploaded['상하한가.csv']), encoding='utf-8', thousands = ',')
df

,종목명,Unnamed: 1,선물옵션\n구분,Unnamed: 3,최종\n거래일,Unnamed: 5,행사가격,Unnamed: 7,기초자산\n기준가격,Unnamed: 9,...,Unnamed: 13,금리(%),Unnamed: 15,배당 현재가치,Unnamed: 17,변동성,Unnamed: 19,Unnamed: 20,이론가격,Unnamed: 22
0,"C 202307 480,000( 10)",NaN,콜,NaN,20230713,NaN,480000.0,NaN,607000,NaN,...,NaN,3.75,NaN,0.0,NaN,0.311,NaN,NaN,128529.69,NaN
1,"C 202307 490,000( 10)",NaN,콜,NaN,20230713,NaN,490000.0,NaN,607000,NaN,...,NaN,3.75,NaN,0.0,NaN,0.311,NaN,NaN,118615.86,NaN
2,"C 202307 500,000( 10)",NaN,콜,NaN,20230713,NaN,500000.0,NaN,607000,NaN,...,NaN,3.75,NaN,0.0,NaN,0.311,NaN,NaN,108755.61,NaN
3,"C 202307 520,000( 10)",NaN,콜,NaN,20230713,NaN,520000.0,NaN,607000,NaN,...,NaN,3.75,NaN,0.0,NaN,0.311,NaN,NaN,89370.76,NaN
4,"C 202307 540,000( 10)",NaN,콜,NaN,20230713,NaN,540000.0,NaN,607000,NaN,...,NaN,3.75,NaN,0.0,NaN,0.311,NaN,NaN,70721.38,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159,"P 202406 600,000( 10)",NaN,풋,NaN,20240613,NaN,600000.0,NaN,607000,NaN,...,NaN,3.75,NaN,0.0,NaN,0.279,NaN,NaN,52525.46,NaN
160,"P 202406 640,000( 10)",NaN,풋,NaN,20240613,NaN,640000.0,NaN,607000,NaN,...,NaN,3.75,NaN,0.0,NaN,0.279,NaN,NaN,72760.65,NaN
161,"P 202406 680,000( 10)",NaN,풋,NaN,20240613,NaN,680000.0,NaN,607000,NaN,...,NaN,3.75,NaN,0.0,NaN,0.279,NaN,NaN,96422.87,NaN
162,"P 202406 720,000( 10)",NaN,풋,NaN,20240613,NaN,720000.0,NaN,607000,NaN,...,NaN,3.75,NaN,0.0,NaN,0.279,NaN,NaN,123649.71,NaN


In [193]:
# null 값 제거
df= df.dropna(subset=['종목명'])

In [194]:
# 금리 단위 맞추기
df['금리(%)']= df['금리(%)']/100

In [195]:
# 풋옵션 이론가 산출 파라미터만 남기기
df2 = df[df['선물옵션\n구분'] == '풋']

In [196]:
# 콜옵션 이론가 산출 파라미터만 남기기
df = df[df['선물옵션\n구분'] == '콜']

#### 1. 콜옵션 이론가 구하기

In [197]:
# null값 제거
df = df.dropna(axis = 1)
df

,종목명,선물옵션\n구분,최종\n거래일,행사가격,기초자산\n기준가격,잔존일수,금리(%),배당 현재가치,변동성,이론가격
0,"C 202307 480,000( 10)",콜,20230713,480000.0,607000,30,0.0375,0.0,0.311,128529.69
1,"C 202307 490,000( 10)",콜,20230713,490000.0,607000,30,0.0375,0.0,0.311,118615.86
2,"C 202307 500,000( 10)",콜,20230713,500000.0,607000,30,0.0375,0.0,0.311,108755.61
3,"C 202307 520,000( 10)",콜,20230713,520000.0,607000,30,0.0375,0.0,0.311,89370.76
4,"C 202307 540,000( 10)",콜,20230713,540000.0,607000,30,0.0375,0.0,0.311,70721.38
...,...,...,...,...,...,...,...,...,...,...
77,"C 202406 600,000( 10)",콜,20240613,600000.0,607000,366,0.0375,0.0,0.311,89096.71
78,"C 202406 640,000( 10)",콜,20240613,640000.0,607000,366,0.0375,0.0,0.311,70953.44
79,"C 202406 680,000( 10)",콜,20240613,680000.0,607000,366,0.0375,0.0,0.311,56532.22
80,"C 202406 720,000( 10)",콜,20240613,720000.0,607000,366,0.0375,0.0,0.311,44698.41


In [198]:
# 이론가격을 구하기 위해 data 자료타입 변경

S = list(map(int,df['기초자산\n기준가격'].tolist()))
K = list(map(int,df['행사가격'].tolist()))
r = list(map (float,df['금리(%)'].tolist()))
div = list(map(int,df['배당  현재가치'].tolist()))
sigma = list(map(float,df['변동성'].tolist()))
t = list(map(int,df['잔존일수'].tolist()))
theo_prc_sys = list(map(float,df['이론가격'].tolist()))
name = df['종목명'].tolist()

In [199]:
# 상하한가 계산을 위해 기초자산 가격을 가격제한폭으로 설정하고 자료타입 변경
upper_lmt =  df['기초자산\n기준가격']*1.3
lower_lmt =  df['기초자산\n기준가격']*0.7
upper_lmt = list(map(int,upper_lmt.tolist()))
lower_lmt = list(map(int,lower_lmt.tolist()))

In [200]:
# call 옵션 이론가격 구하는 코드

def call(s,k,R,T,Div, Sigma):
  u = np.exp(Sigma*np.sqrt(T/365/49))
  d = np.exp(-Sigma*np.sqrt(T/365/49))
  p = (np.exp(R *T/365/49) - d)/(u-d)
  q = 1-p
  y = []

  for i in range(50):
    C_i = max((s-Div) * u**i * d**(49-i) -k , 0)
    x_i = np.math.factorial(49)/(np.math.factorial(49-i)*np.math.factorial(i))
    p_i = p**i
    q_i = q**(49-i)
    y.append(C_i*x_i* p_i*q_i)

  return sum(y)*np.exp(-R*T/365)

In [201]:
# 모든 행사가격, 모든 월물에 대해 이론가격 구하는 코드
call_price_temp = []

for i in range(len(name)):
  s = S[i]
  k = K[i]
  R = r[i]
  T = t[i]
  Div = div[i]
  Sigma = sigma[i]

  price =  call(s,k,R,T,Div, Sigma)
  call_price_temp.append(price)


In [202]:
# 상한가 구하기
upper_lmt_temp = []

for i in range(len(name)):
  s = upper_lmt[i]
  k = K[i]
  R = r[i]
  T = t[i]
  Div = div[i]
  Sigma = sigma[i]

  upper_price =  call(s,k,R,T,Div, Sigma)
  upper_lmt_temp.append(upper_price)


In [203]:
# 하한가 구하기
lower_lmt_temp =[]

for i in range(len(name)):
  s = lower_lmt[i]
  k = K[i]
  R = r[i]
  T = t[i]
  Div = div[i]
  Sigma = sigma[i]

  lower_price = call(s,k,R,T,Div, Sigma)
  lower_lmt_temp.append(lower_price)

In [204]:
# 구한 이론가와 시스템에서 산출한 이론가 비교, 상하한가 표출

call_price = pd.Series(call_price_temp)
upper_limit = pd.Series(upper_lmt_temp)
lower_limit = pd.Series(lower_lmt_temp)
df['theo_price_검증용'] = call_price
df['시스템 검증 차이'] = df['이론가격'] - df['theo_price_검증용']
df['상한가'] = upper_limit
df['하한가'] = lower_limit

In [205]:
df

,종목명,선물옵션\n구분,최종\n거래일,행사가격,기초자산\n기준가격,잔존일수,금리(%),배당 현재가치,변동성,이론가격,theo_price_검증용,시스템 검증 차이,상한가,하한가
0,"C 202307 480,000( 10)",콜,20230713,480000.0,607000,30,0.0375,0.0,0.311,128529.69,128529.691865,-0.001865,310577.174424,1700.803930
1,"C 202307 490,000( 10)",콜,20230713,490000.0,607000,30,0.0375,0.0,0.311,118615.86,118615.858076,0.001924,300607.948930,994.596777
2,"C 202307 500,000( 10)",콜,20230713,500000.0,607000,30,0.0375,0.0,0.311,108755.61,108755.605864,0.004136,290638.723631,577.739422
3,"C 202307 520,000( 10)",콜,20230713,520000.0,607000,30,0.0375,0.0,0.311,89370.76,89370.761544,-0.001544,270700.276943,180.824646
4,"C 202307 540,000( 10)",콜,20230713,540000.0,607000,30,0.0375,0.0,0.311,70721.38,70721.381504,-0.001504,250761.878071,45.673400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,"C 202406 600,000( 10)",콜,20240613,600000.0,607000,366,0.0375,0.0,0.311,89096.71,89096.714576,-0.004576,228812.438003,13131.026849
78,"C 202406 640,000( 10)",콜,20240613,640000.0,607000,366,0.0375,0.0,0.311,70953.44,70953.440282,-0.000282,198932.031805,8805.989169
79,"C 202406 680,000( 10)",콜,20240613,680000.0,607000,366,0.0375,0.0,0.311,56532.22,56532.215320,0.004680,171815.126878,6058.706054
80,"C 202406 720,000( 10)",콜,20240613,720000.0,607000,366,0.0375,0.0,0.311,44698.41,44698.410510,-0.000510,147684.705044,4130.811118


In [206]:
print('시스템 이론가격과 검증이론가격의 차이가 1원 이상 나는 종목')
df[df['시스템 검증 차이'].abs() > 1]

시스템 이론가격과 검증이론가격의 차이가 1원 이상 나는 종목


,종목명,선물옵션\n구분,최종\n거래일,행사가격,기초자산\n기준가격,잔존일수,금리(%),배당 현재가치,변동성,이론가격,theo_price_검증용,시스템 검증 차이,상한가,하한가


#### 2. 풋옵션 이론가 구하기

In [207]:
# null값 제거
df2 = df2.dropna(axis = 1)
df2

,종목명,선물옵션\n구분,최종\n거래일,행사가격,기초자산\n기준가격,잔존일수,금리(%),배당 현재가치,변동성,이론가격
82,"P 202307 480,000( 10)",풋,20230713,480000.0,607000,30,0.0375,0.0,0.279,15.80
83,"P 202307 490,000( 10)",풋,20230713,490000.0,607000,30,0.0375,0.0,0.279,38.93
84,"P 202307 500,000( 10)",풋,20230713,500000.0,607000,30,0.0375,0.0,0.279,86.97
85,"P 202307 520,000( 10)",풋,20230713,520000.0,607000,30,0.0375,0.0,0.279,399.50
86,"P 202307 540,000( 10)",풋,20230713,540000.0,607000,30,0.0375,0.0,0.314,2126.91
...,...,...,...,...,...,...,...,...,...,...
159,"P 202406 600,000( 10)",풋,20240613,600000.0,607000,366,0.0375,0.0,0.279,52525.46
160,"P 202406 640,000( 10)",풋,20240613,640000.0,607000,366,0.0375,0.0,0.279,72760.65
161,"P 202406 680,000( 10)",풋,20240613,680000.0,607000,366,0.0375,0.0,0.279,96422.87
162,"P 202406 720,000( 10)",풋,20240613,720000.0,607000,366,0.0375,0.0,0.279,123649.71


In [208]:
df2.reset_index(inplace = True)

In [209]:
df2

,index,종목명,선물옵션\n구분,최종\n거래일,행사가격,기초자산\n기준가격,잔존일수,금리(%),배당 현재가치,변동성,이론가격
0,82,"P 202307 480,000( 10)",풋,20230713,480000.0,607000,30,0.0375,0.0,0.279,15.80
1,83,"P 202307 490,000( 10)",풋,20230713,490000.0,607000,30,0.0375,0.0,0.279,38.93
2,84,"P 202307 500,000( 10)",풋,20230713,500000.0,607000,30,0.0375,0.0,0.279,86.97
3,85,"P 202307 520,000( 10)",풋,20230713,520000.0,607000,30,0.0375,0.0,0.279,399.50
4,86,"P 202307 540,000( 10)",풋,20230713,540000.0,607000,30,0.0375,0.0,0.314,2126.91
...,...,...,...,...,...,...,...,...,...,...,...
77,159,"P 202406 600,000( 10)",풋,20240613,600000.0,607000,366,0.0375,0.0,0.279,52525.46
78,160,"P 202406 640,000( 10)",풋,20240613,640000.0,607000,366,0.0375,0.0,0.279,72760.65
79,161,"P 202406 680,000( 10)",풋,20240613,680000.0,607000,366,0.0375,0.0,0.279,96422.87
80,162,"P 202406 720,000( 10)",풋,20240613,720000.0,607000,366,0.0375,0.0,0.279,123649.71


In [210]:
# 이론가격을 구하기 위해 data 자료구조 변경

S = list(map(int,df2['기초자산\n기준가격'].tolist()))
K = list(map(int,df2['행사가격'].tolist()))
r = list(map (float,df2['금리(%)'].tolist()))
div = list(map(int,df2['배당  현재가치'].tolist()))
sigma = list(map(float,df2['변동성'].tolist()))
t = list(map(int,df2['잔존일수'].tolist()))
theo_prc_sys = list(map(float,df2['이론가격'].tolist()))
name = df2['종목명'].tolist()

In [211]:
# 상하한가 계산을 위해 기초자산 가격을 가격제한폭으로 설정하고 자료타입 변경
upper_lmt =  df['기초자산\n기준가격']*0.7
lower_lmt =  df['기초자산\n기준가격']*1.3
upper_lmt = list(map(int,upper_lmt.tolist()))
lower_lmt = list(map(int,lower_lmt.tolist()))

In [212]:
# put 옵션 이론가격 구하는 코드

def put(s,k,R,T,Div, Sigma):
  u = np.exp(Sigma*np.sqrt(T/365/49))
  d = np.exp(-Sigma*np.sqrt(T/365/49))
  p = (np.exp(R *T/365/49) - d)/(u-d)
  q = 1-p
  y = []

  for i in range(50):
    P_i = max(k -(s-Div)* u**i * d**(49-i)  , 0)
    x_i = np.math.factorial(49)/(np.math.factorial(49-i)*np.math.factorial(i))
    p_i = p**i
    q_i = q**(49-i)
    y.append(P_i*x_i* p_i*q_i)

  return sum(y)*np.exp(-R*T/365)

In [213]:
# 모든 행사가격, 모든 월물에 대해 이론가격 구하는 코드

put_price_temp = []

for i in range(len(name)):
  s = S[i]
  k = K[i]
  R = r[i]
  T = t[i]
  Div = div[i]
  Sigma = sigma[i]

  price =  put(s,k,R,T,Div,Sigma)
  put_price_temp.append(price)


In [214]:
# 상한가 구하기
upper_lmt_temp = []

for i in range(len(name)):
  s = upper_lmt[i]
  k = K[i]
  R = r[i]
  T = t[i]
  Div = div[i]
  Sigma = sigma[i]

  upper_price =  put(s,k,R,T,Div, Sigma)
  upper_lmt_temp.append(upper_price)

In [215]:
# 하한가 구하기
lower_lmt_temp =[]

for i in range(len(name)):
  s = lower_lmt[i]
  k = K[i]
  R = r[i]
  T = t[i]
  Div = div[i]
  Sigma = sigma[i]

  lower_price = put(s,k,R,T,Div, Sigma)
  lower_lmt_temp.append(lower_price)

In [216]:
# 구한 이론가와 시스템에서 산출한 이론가 비교, 상하한가 표출

call_price = pd.Series(put_price_temp)
upper_limit = pd.Series(upper_lmt_temp)
lower_limit = pd.Series(lower_lmt_temp)
df2['theo_price_검증용'] = call_price
df2['시스템 검증 차이'] = df2['이론가격'] - df2['theo_price_검증용']
df2['상한가'] = upper_limit
df2['하한가'] = lower_limit

In [217]:
df2

,index,종목명,선물옵션\n구분,최종\n거래일,행사가격,기초자산\n기준가격,잔존일수,금리(%),배당 현재가치,변동성,이론가격,theo_price_검증용,시스템 검증 차이,상한가,하한가
0,82,"P 202307 480,000( 10)",풋,20230713,480000.0,607000,30,0.0375,0.0,0.279,15.80,15.804263,-0.004263,54698.785611,1.880150e-08
1,83,"P 202307 490,000( 10)",풋,20230713,490000.0,607000,30,0.0375,0.0,0.279,38.93,38.930755,-0.000755,64183.232300,2.786515e-07
2,84,"P 202307 500,000( 10)",풋,20230713,500000.0,607000,30,0.0375,0.0,0.279,86.97,86.970947,-0.000947,73871.858932,2.925890e-06
3,85,"P 202307 520,000( 10)",풋,20230713,520000.0,607000,30,0.0375,0.0,0.279,399.50,399.504403,-0.004403,93572.653027,1.351270e-04
4,86,"P 202307 540,000( 10)",풋,20230713,540000.0,607000,30,0.0375,0.0,0.314,2126.91,2126.913943,-0.003943,113489.973466,6.693997e-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,159,"P 202406 600,000( 10)",풋,20240613,600000.0,607000,366,0.0375,0.0,0.279,52525.46,52525.456254,0.003746,162324.900913,1.233712e+04
78,160,"P 202406 640,000( 10)",풋,20240613,640000.0,607000,366,0.0375,0.0,0.279,72760.65,72760.654375,-0.004375,197530.597193,1.993439e+04
79,161,"P 202406 680,000( 10)",풋,20240613,680000.0,607000,366,0.0375,0.0,0.279,96422.87,96422.872656,-0.002656,233815.197829,3.040350e+04
80,162,"P 202406 720,000( 10)",풋,20240613,720000.0,607000,366,0.0375,0.0,0.279,123649.71,123649.706803,0.003197,270831.030554,4.340926e+04


In [218]:
print('시스템 이론가격과 검증이론가격의 차이가 1원 이상 나는 종목')
df2[df2['시스템 검증 차이'].abs() > 1]

시스템 이론가격과 검증이론가격의 차이가 1원 이상 나는 종목


,index,종목명,선물옵션\n구분,최종\n거래일,행사가격,기초자산\n기준가격,잔존일수,금리(%),배당 현재가치,변동성,이론가격,theo_price_검증용,시스템 검증 차이,상한가,하한가
